# Building Custom Skills for Claude

Learn how to create, deploy, and manage custom skills to extend Claude's capabilities with your organization's specialized knowledge and workflows.

## Table of Contents

1. [Introduction & Setup](#introduction)
2. [Understanding Custom Skills Architecture](#architecture)
3. [Example 1: Financial Ratio Calculator](#financial-ratio)
4. [Example 2: Company Brand Guidelines](#brand-guidelines)
5. [Example 3: Financial Modeling Suite](#financial-modeling)
6. [Skill Management & Versioning](#management)
7. [Best Practices & Production Tips](#best-practices)
8. [Troubleshooting](#troubleshooting)

## 1. Introduction & Setup {#introduction}

### What are Custom Skills?

**Custom skills** are specialized expertise packages you create to teach Claude your organization's unique workflows, domain knowledge, and best practices. Unlike Anthropic's pre-built skills (Excel, PowerPoint, PDF), custom skills allow you to:

- **Codify organizational knowledge** - Capture your team's specific methodologies
- **Ensure consistency** - Apply the same standards across all interactions
- **Automate complex workflows** - Chain together multi-step processes
- **Maintain intellectual property** - Keep proprietary methods secure

### Key Benefits

| Benefit | Description |
|---------|-------------|
| **Expertise at Scale** | Deploy specialized knowledge to every Claude interaction |
| **Version Control** | Track changes and roll back if needed |
| **Composability** | Combine multiple skills for complex tasks |
| **Privacy** | Your skills remain private to your organization |

### Prerequisites

Before starting, ensure you have:
- Completed [Notebook 1: Introduction to Skills](01_skills_introduction.ipynb)
- An Anthropic API key with Skills beta access
- Python environment with the local SDK installed

### Environment Setup

Let's set up our environment and import necessary libraries:

In [ ]:
import os
import sys
import json
import shutil
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

# Add parent directory for imports
sys.path.insert(0, str(Path.cwd().parent))

from anthropic import Anthropic
from anthropic.lib import files_from_dir
from dotenv import load_dotenv

# Import our utilities
from file_utils import (
    extract_file_ids,
    download_all_files,
    get_file_info,
    print_download_summary
)

# We'll create skill_utils later in this notebook
# from skill_utils import (
#     create_skill,
#     list_skills,
#     delete_skill,
#     test_skill
# )

# Load environment variables
load_dotenv(Path.cwd().parent / '.env')

API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-5-20250929")

if not API_KEY:
    raise ValueError(
        "ANTHROPIC_API_KEY not found. "
        "Copy ../.env.example to ../.env and add your API key."
    )

# Initialize client with Skills beta
client = Anthropic(
    api_key=API_KEY,
    default_headers={
        "anthropic-beta": "skills-2025-10-02"
    }
)

# Setup directories
SKILLS_DIR = Path.cwd().parent / "custom_skills"
OUTPUT_DIR = Path.cwd().parent / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ API key loaded")
print(f"✓ Using model: {MODEL}")
print(f"✓ Custom skills directory: {SKILLS_DIR}")
print(f"✓ Output directory: {OUTPUT_DIR}")
print("\n📝 Skills beta header configured for skill management")

## 2. Understanding Custom Skills Architecture {#architecture}

### Skill Structure

Every custom skill must follow this directory structure:

```
skill_name/
├── SKILL.md           # Required: Instructions with YAML frontmatter
├── REFERENCE.md       # Optional: Additional documentation
├── scripts/           # Optional: Executable code
│   ├── process.py
│   └── utils.js
└── resources/         # Optional: Templates, data files
    └── template.xlsx
```

### SKILL.md Requirements

The `SKILL.md` file must include:

1. **YAML Frontmatter** (max 1024 chars)
   - `name`: Lowercase alphanumeric with hyphens
   - `description`: Brief description of what the skill does

2. **Instructions** (markdown format)
   - Clear guidance for Claude
   - Examples of usage
   - Any constraints or rules

### Progressive Disclosure

Skills load in three stages to optimize token usage:

| Stage | Content | Token Cost | When Loaded |
|-------|---------|------------|-------------|
| **1. Metadata** | Name & description | ~100 tokens | Always visible |
| **2. Instructions** | SKILL.md content | <5,000 tokens | When relevant |
| **3. Resources** | Scripts & files | As needed | During execution |

### API Workflow

```python
# 1. Create skill
skill = client.beta.skills.create(
    display_title="My Skill",
    files=files_from_dir("path/to/skill")
)

# 2. Use in messages
response = client.beta.messages.create(
    container={
        "skills": [{
            "type": "custom",
            "skill_id": skill.id,
            "version": "latest"
        }]
    },
    # ... rest of message parameters
)
```

### Create Skill Utility Functions

Let's create helper functions for skill management:

In [ ]:
def create_skill(
    client: Anthropic,
    skill_path: str,
    display_title: str
) -> Dict[str, Any]:
    """
    Create a new custom skill from a directory.
    
    Args:
        client: Anthropic client instance
        skill_path: Path to skill directory
        display_title: Human-readable skill name
        
    Returns:
        Dictionary with skill_id, version, and metadata
    """
    try:
        # Create skill using files_from_dir
        skill = client.beta.skills.create(
            display_title=display_title,
            files=files_from_dir(skill_path)
        )
        
        return {
            'success': True,
            'skill_id': skill.id,
            'display_title': skill.display_title,
            'latest_version': skill.latest_version,
            'created_at': skill.created_at,
            'source': skill.source
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }


def list_custom_skills(client: Anthropic) -> List[Dict[str, Any]]:
    """
    List all custom skills in the workspace.
    
    Returns:
        List of skill dictionaries
    """
    try:
        skills_response = client.beta.skills.list(source="custom")
        
        skills = []
        for skill in skills_response.data:
            skills.append({
                'skill_id': skill.id,
                'display_title': skill.display_title,
                'latest_version': skill.latest_version,
                'created_at': skill.created_at,
                'updated_at': skill.updated_at
            })
        
        return skills
    except Exception as e:
        print(f"Error listing skills: {e}")
        return []


def delete_skill(client: Anthropic, skill_id: str) -> bool:
    """
    Delete a custom skill and all its versions.
    
    Args:
        client: Anthropic client
        skill_id: ID of skill to delete
        
    Returns:
        True if successful, False otherwise
    """
    try:
        # First delete all versions
        versions = client.beta.skills.versions.list(skill_id=skill_id)
        
        for version in versions.data:
            client.beta.skills.versions.delete(
                skill_id=skill_id,
                version=version.version
            )
        
        # Then delete the skill itself
        client.beta.skills.delete(skill_id)
        return True
        
    except Exception as e:
        print(f"Error deleting skill: {e}")
        return False


def test_skill(
    client: Anthropic,
    skill_id: str,
    test_prompt: str,
    model: str = "claude-sonnet-4-5-20250929"
) -> Any:
    """
    Test a custom skill with a prompt.
    
    Args:
        client: Anthropic client
        skill_id: ID of skill to test
        test_prompt: Prompt to test the skill
        model: Model to use for testing
        
    Returns:
        Response from Claude
    """
    response = client.beta.messages.create(
        model=model,
        max_tokens=4096,
        container={
            "skills": [{
                "type": "custom",
                "skill_id": skill_id,
                "version": "latest"
            }]
        },
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{"role": "user", "content": test_prompt}],
        betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
    )
    
    return response


print("✓ Skill utility functions defined")
print("  - create_skill()")
print("  - list_custom_skills()")
print("  - delete_skill()")
print("  - test_skill()")

### Check Existing Custom Skills

Let's see if any custom skills already exist in your workspace:

In [ ]:
# List existing custom skills
existing_skills = list_custom_skills(client)

if existing_skills:
    print(f"Found {len(existing_skills)} existing custom skill(s):\n")
    for skill in existing_skills:
        print(f"📦 {skill['display_title']}")
        print(f"   ID: {skill['skill_id']}")
        print(f"   Version: {skill['latest_version']}")
        print(f"   Created: {skill['created_at']}\n")
else:
    print("No custom skills found. Let's create some!")

## 3. Example 1: Financial Ratio Calculator {#financial-ratio}

Let's create our first custom skill - a financial ratio calculator that can analyze company financial health.

### Skill Overview

The **Financial Ratio Calculator** skill will:
- Calculate key financial ratios (ROE, P/E, Current Ratio, etc.)
- Interpret ratios with industry context
- Generate formatted reports
- Work with various data formats (CSV, JSON, text)

### Upload the Financial Analyzer Skill

Now let's upload our financial analyzer skill to Claude:

In [ ]:
# Upload the Financial Analyzer skill
financial_skill_path = SKILLS_DIR / "financial_analyzer"

if financial_skill_path.exists():
    print("Uploading Financial Analyzer skill...")
    result = create_skill(
        client,
        str(financial_skill_path),
        "Financial Ratio Analyzer"
    )
    
    if result['success']:
        financial_skill_id = result['skill_id']
        print(f"✅ Skill uploaded successfully!")
        print(f"   Skill ID: {financial_skill_id}")
        print(f"   Version: {result['latest_version']}")
        print(f"   Created: {result['created_at']}")
    else:
        print(f"❌ Upload failed: {result['error']}")
else:
    print(f"⚠️ Skill directory not found: {financial_skill_path}")
    print("Please ensure the custom_skills directory contains the financial_analyzer folder.")

### Test the Financial Analyzer Skill

Let's test the skill with sample financial data:

In [ ]:
# Test the Financial Analyzer skill
if 'financial_skill_id' in locals():
    test_prompt = """
    Calculate financial ratios for this company:
    
    Income Statement:
    - Revenue: $1,000M
    - EBITDA: $200M
    - Net Income: $120M
    
    Balance Sheet:
    - Total Assets: $2,000M
    - Current Assets: $500M
    - Current Liabilities: $300M
    - Total Debt: $400M
    - Shareholders Equity: $1,200M
    
    Market Data:
    - Share Price: $50
    - Shares Outstanding: 100M
    
    Please calculate key ratios and provide analysis.
    """
    
    print("Testing Financial Analyzer skill...")
    response = test_skill(client, financial_skill_id, test_prompt)
    
    # Print response
    for content in response.content:
        if content.type == "text":
            print(content.text)
else:
    print("⚠️ Please upload the Financial Analyzer skill first (run the previous cell)")

## 4. Example 2: Company Brand Guidelines {#brand-guidelines}

Now let's create a skill that ensures all documents follow corporate brand standards.

### Skill Overview

The **Brand Guidelines** skill will:
- Apply consistent colors, fonts, and layouts
- Ensure logo placement and usage
- Maintain professional tone and messaging
- Work across all document types (Excel, PowerPoint, PDF)

In [ ]:
# Upload the Brand Guidelines skill
brand_skill_path = SKILLS_DIR / "brand_guidelines"

if brand_skill_path.exists():
    print("Uploading Brand Guidelines skill...")
    result = create_skill(
        client,
        str(brand_skill_path),
        "Corporate Brand Guidelines"
    )
    
    if result['success']:
        brand_skill_id = result['skill_id']
        print(f"✅ Skill uploaded successfully!")
        print(f"   Skill ID: {brand_skill_id}")
        print(f"   Version: {result['latest_version']}")
    else:
        print(f"❌ Upload failed: {result['error']}")
else:
    print(f"⚠️ Skill directory not found: {brand_skill_path}")

### Test Brand Guidelines with Document Creation

Let's test the brand skill by creating a branded PowerPoint presentation:

In [ ]:
# Test Brand Guidelines skill with PowerPoint creation
if 'brand_skill_id' in locals():
    # Combine brand skill with Anthropic's pptx skill
    response = client.beta.messages.create(
        model=MODEL,
        max_tokens=4096,
        container={
            "skills": [
                {"type": "custom", "skill_id": brand_skill_id, "version": "latest"},
                {"type": "anthropic", "skill_id": "pptx", "version": "latest"}
            ]
        },
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{
            "role": "user",
            "content": """Create a 3-slide PowerPoint presentation following Acme Corporation brand guidelines:
            
            Slide 1: Title slide for "Q4 2025 Results"
            Slide 2: Revenue Overview with a chart showing Q1-Q4 growth
            Slide 3: Key Achievements (3 bullet points)
            
            Apply all brand colors, fonts, and formatting standards.
            """
        }],
        betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
    )
    
    print("Response from Claude:")
    for content in response.content:
        if content.type == "text":
            print(content.text[:500] + "..." if len(content.text) > 500 else content.text)
    
    # Download generated file
    file_ids = extract_file_ids(response)
    if file_ids:
        results = download_all_files(client, response, output_dir=str(OUTPUT_DIR), prefix="branded_")
        print_download_summary(results)
else:
    print("⚠️ Please upload the Brand Guidelines skill first")

## 7. Best Practices & Production Tips {#best-practices}

### Skill Design Principles

1. **Single Responsibility**: Each skill should focus on one area of expertise
2. **Clear Documentation**: SKILL.md should be comprehensive yet concise
3. **Error Handling**: Scripts should handle edge cases gracefully
4. **Version Control**: Use Git to track skill changes
5. **Testing**: Always test skills before production deployment

### Directory Structure Best Practices

```
custom_skills/
├── financial_analyzer/       # Single purpose, clear naming
│   ├── SKILL.md             # Under 5,000 tokens
│   ├── scripts/             # Modular Python/JS files
│   └── tests/               # Unit tests for scripts
├── brand_guidelines/         # Organizational standards
│   ├── SKILL.md
│   ├── REFERENCE.md         # Additional documentation
│   └── assets/              # Logos, templates
```

### Performance Optimization

| Strategy | Impact | Implementation |
|----------|--------|----------------|
| **Minimal Frontmatter** | Faster skill discovery | Keep under 500 chars |
| **Lazy Loading** | Reduced token usage | Reference files only when needed |
| **Skill Composition** | Avoid duplication | Combine skills vs. mega-skill |
| **Caching** | Faster responses | Reuse skill containers |

### Security Considerations

- **API Keys**: Never hardcode credentials in skills
- **Data Privacy**: Don't include sensitive data in skill files
- **Access Control**: Skills are workspace-specific
- **Validation**: Sanitize inputs in scripts
- **Audit Trail**: Log skill usage for compliance

## 8. Troubleshooting {#troubleshooting}

### Common Issues and Solutions

#### Skill Upload Errors

**Error: "SKILL.md not found"**
```python
# Solution: Ensure SKILL.md exists at root of skill directory
skill_dir/
├── SKILL.md  # Required at this exact location
└── ...
```

**Error: "YAML frontmatter exceeds 1024 characters"**
```yaml
---
name: my-skill  # Keep concise
description: Brief description under 100 chars  # Shorter is better
---
```

**Error: "Invalid skill name"**
```yaml
---
name: my-skill-name  # ✅ Lowercase, hyphens only
name: MySkillName    # ❌ No capitals
name: my_skill       # ❌ No underscores
---
```

#### Skill Execution Issues

**Skills not loading in messages**
```python
# Ensure you're using beta.messages.create()
response = client.beta.messages.create(  # ✅ beta namespace
    container={"skills": [...]},
    # ...
)
```

**Code execution not working**
```python
# Must include code_execution tool with skills
tools=[{"type": "code_execution_20250825", "name": "code_execution"}]
```

#### Version Management Issues

**Can't delete skill**
```python
# Delete all versions first
versions = client.beta.skills.versions.list(skill_id=skill_id)
for v in versions.data:
    client.beta.skills.versions.delete(skill_id=skill_id, version=v.version)
# Then delete skill
client.beta.skills.delete(skill_id)
```

### Debugging Tips

1. **Check skill structure**: Use validation function before upload
2. **Test incrementally**: Start with simple skill, add complexity
3. **Monitor token usage**: Log token counts for optimization
4. **Use version control**: Track what changed between versions
5. **Enable verbose logging**: Capture full API responses

### Getting Help

- **Documentation**: Review Skills API User Guide
- **Support**: Contact Anthropic support for API issues
- **Community**: Share experiences and solutions
- **Updates**: Check for SDK updates regularly

## Next Steps

🎉 **Congratulations!** You've learned how to create, deploy, and manage custom skills for Claude.

### What You've Learned

- ✅ Custom skill architecture and requirements
- ✅ Creating skills with SKILL.md and Python scripts
- ✅ Uploading skills via the API
- ✅ Combining custom and Anthropic skills
- ✅ Best practices for production deployment
- ✅ Troubleshooting common issues

### Continue Your Journey

1. **Experiment**: Modify the example skills for your use cases
2. **Build**: Create skills for your organization's workflows
3. **Optimize**: Monitor token usage and performance
4. **Share**: Document your skills for team collaboration

### Resources

- [Claude API Documentation](https://docs.anthropic.com/en/api/messages)
- [Skills Documentation](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/skills)
- [Files API Documentation](https://docs.claude.com/en/api/files-content)
- Example Skills Repository (coming soon)

### Skill Ideas to Try

- 📊 **Data Pipeline**: ETL workflows with validation
- 📝 **Document Templates**: Contracts, proposals, reports
- 🔍 **Code Review**: Style guides and best practices
- 📈 **Analytics Dashboard**: KPI tracking and visualization
- 🤖 **Automation Suite**: Repetitive task workflows

Happy skill building! 🚀